# Cell Cell communication - ligand-receptor inference

This notebook was designed based off of chapter 21 of the book [Single-cell best practices](https://www.sc-best-practices.org/preamble.html) after Heumos, L., Schaar, A.C., Lance, C. et al. Best practices for single-cell analysis across modalities. Nat Rev Genet (2023). https://doi.org/10.1038/s41576-023-00586-w 

## Environment setup

Set variables and load packages 

In [ ]:
#################################################### SET VARIABLES #####################################################

# Path to h5ad file for experiment
path_in = "../data/"

# Path to save results to
path_out = "../results/new_29_01_25/M26/"

# Set the experiment prefix to be added to all names of saved result files
experiment_prefix = "M26"

# Set name up input .h5ad file
name_adata = "MH112_adata_cyno_final_CCC.h5ad"

# Name of layer containing raw data counts
# If set to "X", the raw data will be saved in the anndata.raw layer and X normalized after
# If set to "raw", the raw layer will be used for normalization and X saved in a layer called ".X" for later restoration
# Any other value will assume it is the name of anndata.layers[layer_raw] where the raw data is saved. Here too, the X matrix is stored in layer ".X" for later restoration
layer_raw = "X"

# Name of column in obs table that contains the conditions/timepoints of the experiment
condition_col = "timepoint"

# Name of column in obs table that contains the clusters/cell types to be compared
cluster_col = "cluster_azimut1_5_scanvi_v2"

# Name of column with cell types in cell type annotation file
celltype_col = "cluster_azimut1_5_scanvi_v2"

# Path in cell type annotation
path_in_celltype_anno = "../data/MH70_cluster_anno.csv"

##########################################################################################################################

In [ ]:
# Import Python packages

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.graph_objects as go
from plotnine import *
import warnings
import os
import anndata as ad
import glob

import scanpy as sc
import liana as li
from liana.method import rank_aggregate

import session_info

In [ ]:
# import helper functions 
import importlib.util

spec = importlib.util.spec_from_file_location("helpers", "../helpers/helpersCCC.py")
helpers = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helpers)

In [ ]:
# Setting up R dependencies

import anndata2ri 
import rpy2 
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
import random

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
%%R

# Import R libraries
suppressPackageStartupMessages({
    library(circlize)
    library(data.table)
    library(stringr)
    library(reticulate) 
    library(ggplot2) 
    library(tidyr)
    library(dplyr) 
    library(tibble)
    source("../helpers/helpersCCC.R")
})

# Load data and data preparation

Load experiment as AnnData object from .h5ad file and get an overview of the data.

In [ ]:
# Set full path to input file
path_in_adata = os.path.join(path_in, name_adata)

# Read in h5ad for experiment as AnnData object
adata = sc.read_h5ad(path_in_adata)

# Convert celltype and group colums to dtype categorial
adata.obs[condition_col] = adata.obs[condition_col].astype("category")

Add new cell type annotation.

In [ ]:
# Read in celltype annotation data frame
celltype_df = pd.read_csv(path_in_celltype_anno, sep=",", index_col=0)

# Add column for cell type annotaion to Anndata
adata.obs[cluster_col] = celltype_df[celltype_col]

# Convert celltype and group colums to dtype categorial
adata.obs[cluster_col] = adata.obs[cluster_col].astype("category")

Get an overview of the loaded AnnDate object.

In [ ]:
adata

In [ ]:
adata.obs

In [ ]:
adata.var

### Normalize data

Normalize raw data passed with AnnData object and set as new .X matrix. Do this to make sure inference methods work with data normalized in a way they can handle.

In [ ]:
# If .X does not contain raw counts, save .X to layers for later restoration
if layer_raw == "X":
    # Save the raw data in the raw layer and continue
    adata.raw = adata.copy()

elif layer_raw == "raw":
    # Save X matrix in layer ".X" and set raw data as new X matrix
    adata.layers[".X"] = adata.X.copy() 
    adata.X = adata.raw.X.copy()
    
else:
    # If any other value for layer_raw is given, save X matrix in layer ".X" and
    # Set anndata.layers[layer_raw] as new X matrix
    adata.layers[".X"] = adata.X.copy() 
    adata.X = adata.layers[layer_raw].copy()


#### Visual inspection of data

If an embendding for visualization has not yet been cacluated for the data or has not been passed on, execute the next cell to calculate it now. This may take some time.

In [ ]:
# Normalization, highly variable genes selection, scaling and pca calculation before
# embedding calculation for more Seurat v4 like results

# Save the raw data in the raw layer
adata_copy = adata.copy()

# Normalize, then log1p transform the data
sc.pp.normalize_total(adata_copy)
sc.pp.log1p(adata_copy)

# Identify highly variable genes
sc.pp.highly_variable_genes(adata_copy)
adata_copy = adata_copy[:, adata_copy.var.highly_variable]

# Scale data (equivalent to Seurat's ScaleData)
sc.pp.scale(adata_copy)

# Calculate PCA
sc.pp.pca(adata_copy, n_comps=50)

# Calculate neighbors with Seurat-like parameters
sc.pp.neighbors(adata_copy, 
                n_neighbors=20,      # Seurat default
                n_pcs=30,            # Seurat typically uses 30 PCs
                metric='euclidean',  # Seurat's default
                use_rep='X_pca'      # Ensure using PCA, like Seurat
               )

In [ ]:
# Calculate embedding for data set with parameters finetuned for results closer to Seurat v4

# Calculate UMAP with Seurat-like parameters
sc.tl.umap(adata_copy, 
           min_dist=0.3,    # Seurat default=0.3
           spread=0.5       # Seurat default=1.0
          )

In [ ]:
# Calculate tSNE with Seurat-like parameters
# NOTE: Seurat uses a different perplexity calculation, but we can set it to 30
sc.tl.tsne(adata_copy,
           n_pcs=30,           # Seurat typically uses 30 PCs
           perplexity=30,      # Seurat default is 30
           learning_rate=adata_copy.n_obs/12, # Set to n_cells/12 in Seurat (or ~1000)
           metric='euclidean', # Match Seurat's default
           random_state=42,    # For reproducibility
           n_jobs=8)     

Plot the calculated embeddings.

In [ ]:
# Show UMAP to showcase data
sc.pl.embedding(adata_copy, "X_umap", color=[condition_col, cluster_col], wspace=0.2)

In [ ]:
# Show tSNE to showcase data
sc.pl.embedding(adata_copy, "X_tsne", color=[condition_col, cluster_col], wspace=0.2)

In [ ]:
# Add UMAP and tSNE to the original anndata object
adata.obsm["X_umap"] = adata_copy.obsm["X_umap"].copy()
adata.obsm["X_tsne"] = adata_copy.obsm["X_tsne"].copy()

## Ligand-receptor inference

Inference of CCC events in steady state data using the gene expression of ligand and receptor genes in cell type cluster pairs.

Liana generally works with all expressed genes and gene expression values that have been normalized and log-transformed. Hence this is the first step.


In [ ]:
# Normalize, then log1p transform the data
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

### Generate ligand-receptor inference consensus between different methods using LIANA

The python package liana provides the implementation of 7 different methods to infer CCC events from single-cell gene expression data. These methods can all be applied to the data to generate a magnitude score and a specificity score for each ligand-receptor pair between two cell type clusters respectively. Liana then aggregates these scores into one magnitude and one specificity score per ligand-receptor pair per cell type cluster par for a stronger predictive power.

All methods use prior knowledge resources in regards to ligand-receptor interactions. Liana has aggregated the seperate resources normally used be the different methods into one consesus resource that is used as default. There are a number of different smaller resources liana also provides or a custom resource can be set by the user. 

For more information on the liana package visit the [docs](https://liana-py.readthedocs.io/en/latest/index.html), the [git repo](https://github.com/saezlab/liana-py) or the [paper](https://www.nature.com/articles/s41467-022-30755-0).

In [ ]:
# Get an overview of the methods (re-)implemented by liana, the scores they
# generate and their source
li.method.show_methods()

### Steady state data

Ligand-receptor inference methods as provided by the liana package work with steady state data. This means that if a dataset contains multiple different conditions or timepoints each timepoint must be assessed separately. Additonally, if the dataset contains bataches these should be well corrected.

So, first it is decided which conditions/timepoints within the dataset to perform the ligand-receptor inference on.

In [ ]:
# Get overview of conditions/timepoints in experiment
list(adata.obs[condition_col].cat.categories)

The following code sets all conditions/timepoints in the condition_col to be analyzed. Should you wish to use a subset of condtions/timepoints, set `liana_conds` as follows:

```{python}
liana_conds = ["<condition/timepoint 1>", "<condition/timepoint 2>", ...]
```

In [ ]:
# Give names of conditions/timepoints in conditoin_col of adata.obs table to use for 
# ligand-receptor inference
liana_conds = adata.obs[condition_col].cat.categories

### Score calculation

Calculation of aggregated magnitude and specificity scores per ligand-receptor interaction between cell types

In [ ]:
# Split adata into one object per condition/timepoint and
# calculate aggregated ligand-receptor scores for each condition/timepoint
# This may take a couple of minutes
cond_adatas = {}

for cond in liana_conds:
    print("Caclulating scores for condition/timepoint: " + cond)
    cond_adatas[cond] = adata[adata.obs[condition_col] == cond].copy()
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        rank_aggregate(
            cond_adatas[cond], 
            groupby=cluster_col, 
            return_all_lrs=True, 
            use_raw=False, 
            verbose=True
        )
    
    print("\n")
    display(cond_adatas[cond].uns["liana_res"])
    print("\n")

Add number of cells in cluster information for source clusters and target clsuters.

In [ ]:
for cond in liana_conds:
    # Get number of cells per cluster
    cells_per_cluster = cond_adatas[cond].obs[cluster_col].value_counts().reset_index()
    for col in ["source", "target"]:
        cells_per_cluster_tmp = cells_per_cluster.rename(columns={cluster_col:col})
        new_liana_cond = cond_adatas[cond].uns['liana_res'].merge(cells_per_cluster_tmp, on=col, how='left')
        cols = new_liana_cond.columns.to_list()
        cols.remove('count')
        if col == "source":
            new_liana_cond = new_liana_cond[[cols[0], 'count'] + cols[1:]]
        else:
            new_liana_cond = new_liana_cond[cols[0:3] + ['count'] + cols[3:]]
            
        cond_adatas[cond].uns['liana_res'] = new_liana_cond.rename(columns={'count':'_'.join(['n_cells_in', col, 'cluster'])})

In [ ]:
for cond in liana_conds:
    print(cond)
    display(cond_adatas[cond].uns['liana_res'])

#### Save liana results per condition/timepoint.

In [ ]:
for cond in liana_conds:
    cond_adatas[cond].uns["liana_res"].to_csv(path_or_buf=os.path.join(path_out, "_".join([experiment_prefix, cond, "liana_res.txt.gz"])))

#### Add liana results per condition/timepoint to uns layer of experiment AnnData object.


In [ ]:
for cond in liana_conds:
    adata.uns["liana_res_" + cond] = cond_adatas[cond].uns["liana_res"]

### Dotplots

Generate plots to visualize results of aggregated scores filtered for significance for each condition/timepoint.

In [ ]:
# Get min and max values vor color scale and dot size of plots
max_col = float('-inf')
max_size = float('-inf')

for cond in liana_conds:
    liana_res = cond_adatas[cond].uns["liana_res"]
    cond_max_size = liana_res["specificity_rank"].apply(lambda x: -np.log10(x + np.finfo(float).eps)).max()
    cond_max_col = liana_res["magnitude_rank"].apply(lambda x: -np.log10(x + np.finfo(float).eps)).max()
    
    max_col = cond_max_col if cond_max_col > max_col else max_col
    max_size = cond_max_size if cond_max_size > max_size else max_size

In [ ]:
dotplots = []

for cond in liana_conds:
    # Get cell type labels
    celltypes_sender = adata.uns["liana_res_" + cond]["source"].unique()
    celltypes_receiver = adata.uns["liana_res_" + cond]["target"].unique()

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        # Add custom scaling
        top_n = 30
        fig_width = max(max(len(celltype) for celltype in celltypes_sender), len(celltypes_receiver)) * max(len(celltypes_receiver), len(celltypes_sender)) / 5
        fig_height = top_n // 2

        dotplots.append(
            li.pl.dotplot(
                adata=cond_adatas[cond],
                colour="magnitude_rank",
                size="specificity_rank",
                inverse_colour=True,  # we inverse sign since we want small p-values to have large sizes
                inverse_size=True,
                # We choose only the cell types which we wish to plot
                source_labels=celltypes_sender,
                target_labels=celltypes_receiver,
                # since the rank_aggregate can also be interpreted as a probability distribution
                # we can again filter them according to their specificity significance
                # yet here the interactions are filtered according to
                # how consistently highly-ranked their specificity is across the methods
                filterby="specificity_rank",
                filter_lambda=lambda x: x <= 0.05,
                # We can further order according to magnitude
                orderby="magnitude_rank",
                orderby_ascending=True,  # prioritize those with lowest values
                top_n=top_n,  # and we want to keep only the top 30 interactions
                figure_size=(fig_width, fig_height),
                size_range=(1, 6),
            )
            + ggtitle(cond + " Source")
            + scale_size_continuous(range=(1, 6), limits=[0, max_size])
            + scale_color_cmap('viridis', limits=[0, max_col]))
    
dotplots

#### Save Dotplots.

In [ ]:
for dotplot, cond in zip(dotplots, liana_conds):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        for file_type in [".png", ".pdf"]:
            filename = "_".join([experiment_prefix, cond , "liana_dotplot" + file_type])
            dotplot.save(os.path.join(path_out, filename), dpi=600, limitsize=False)        

The aggregated scores of `magnitude_rank` and `specificity_rank` can be seen as probability distributions, i.e. p-values. Meaning, the smaller the value the higher the significance.

### Circos plots
Generate one circos plot per condition/timepoint that shows the number of detected ligand-receptor interactions between celltypes.

In [ ]:
# Get counts tables for plotting
%R counts_conds <- c()

min_range = 1
max_range = float('-inf')

for cond in liana_conds:
    # Get liana results tables and filter for significance in magnitude and specificity of ligand-receptor interactions
    liana_res_cond = cond_adatas[cond].uns["liana_res"].copy()
    liana_res_cond = liana_res_cond[(liana_res_cond["magnitude_rank"] <= 0.05) & (liana_res_cond["specificity_rank"] <= 0.05)]
    
    # Get number of significant interactions per celltype pair
    counts_cts_cond = liana_res_cond.groupby(["source", "target"]).count()["ligand_complex"].reset_index()
    # Rename colums for R function
    counts_cts_cond_r = counts_cts_cond.rename(columns={"source": "source_cluster", 
                                                        "target": "target_cluster", 
                                                        "ligand_complex": "number"})
    %Rpush counts_cts_cond_r cond
    %R counts_conds[[cond]] <- counts_cts_cond_r

    
    # Update max_range to get experiment wide max number of interactions between a celltype pair
    max_range = max_range if counts_cts_cond_r["number"].max() < max_range else counts_cts_cond_r["number"].max()
    

In [ ]:
%%R -i liana_conds -i min_range -i max_range

for (cond in liana_conds){
    counts_cts_cond_r <- counts_conds[[cond]]
    
    # Plot title
    plot_title <- paste0("CCC between celltypes \nfor condition ", cond)
    
    # Plot cicos plots
    visualize_interactions(counts_cts_cond_r, plot_title=plot_title, color_fill_arrows = c("yellow", "#9933CC"), legendsize = 1.0, min_range=min_range, max_range=max_range)
}

The next cell plots all condition/timepoint circos plots onto a grid for better visual comparing. In the example data there were four conditions, hence we set a 2 x 2 grid for the four plots. Should the number of conditions/timepoints analyzed with LIANA for your data be differen, set `par(mfrwo=c())` accordingly:

```{R}
par(mfrow(<number of rows>, <number of columns>))
```

In [ ]:
%%R -i liana_conds
# Set grid for plots
grid_columns <- 3
grid_rows <- ceiling(length(liana_conds) / 3)

par(mfrow=c(grid_columns, grid_rows))

for (cond in liana_conds) {
  counts_cts_cond_r <- counts_conds[[cond]]
  
  # Set plot title
  plot_title <- paste0("CCC between celltypes \nfor condition ", cond)
  
  # Plot cicos plots
  visualize_interactions(counts_cts_cond_r, plot_title=plot_title, color_fill_arrows = c("yellow", "#9933CC"), legendsize = 1.0, min_range=min_range, max_range=max_range)
}

#### Save circos plots.

Each plot in a separate file.

In [ ]:
%%R -i liana_conds -i path_out -i experiment_prefix
# Save plots separately 
for (cond in liana_conds) {
    counts_cts_cond_r <- counts_conds[[cond]]
  
    # Plot title
    plot_title <- paste0("CCC between celltypes \nfor condition ", cond)

    # Set filepath without fileneding
    filepath <- paste0(path_out, experiment_prefix, "_", cond, "_circos_plot_liana_res")

    # Save as PNG
    png(paste0(filepath, ".png"), units = "in", width=18, height=18, res=600)
    visualize_interactions(counts_cts_cond_r, plot_title=plot_title, color_fill_arrows = c("yellow", "#9933CC"), legendsize = 1.0)
    dev.off()

    # Save as PDF
    pdf(paste0(filepath, ".pdf"), width=18, height=18)
    visualize_interactions(counts_cts_cond_r, plot_title=plot_title, color_fill_arrows = c("yellow", "#9933CC"), legendsize = 1.0)
    dev.off()
}

Or all plots in one file on a grid. Set `mfrow(<number of rows>, <number of columns>)` according to the number of conditions/timepoints in analysed by LIANA.

In [ ]:
%%R -i liana_conds -i min_range -i max_range -i path_out -i experiment_prefix
grid_columns <- 3
grid_rows <- ceiling(length(liana_conds) / 3)

# Save in one graphic as PNG
filepath <- paste0(path_out, experiment_prefix, "_", "All_conds_circos_plot.png")
png(filepath, units = "in", width=45, height=45, res=600)
par(mfrow=c(grid_columns, grid_rows))

for (cond in liana_conds) {
  counts_cts_cond_r <- counts_conds[[cond]]
  
  # Plot title
  plot_title <- paste("CCC between celltypes \nfor condition ", cond)
  
  # Plot cicos plots
  visualize_interactions(counts_cts_cond_r, plot_title=plot_title, color_fill_arrows = c("yellow", "#9933CC"), legendsize = 1.0, min_range=min_range, max_range=max_range)
}
dev.off()


# Save in one graphic as PDF
filepath <- paste0(path_out, experiment_prefix, "_", "All_conds_circos_plot.pdf")
pdf(filepath, width=45, height=45)
par(mfrow=c(grid_columns, grid_rows))
for (cond in liana_conds) {
  counts_cts_cond_r <- counts_conds[[cond]]
  
  # Plot title
  plot_title <- paste("CCC between celltypes \nfor condition ", cond)
  
  # Plot cicos plots
  visualize_interactions(counts_cts_cond_r, plot_title=plot_title, color_fill_arrows = c("yellow", "#9933CC"), legendsize = 1.0, min_range=min_range, max_range=max_range)
}
dev.off()

#### Save tables for plots.

In [ ]:
%%R -i liana_conds -i path_out -i experiment_prefix

for (cond in liana_conds) {
    counts_cts_cond_r <- counts_conds[[cond]]
  
    # Plot title
    plot_title <- paste0("CCC between celltypes \nfor condition ", cond)
    filepath <- paste0(path_out, experiment_prefix, "_", cond, "_circos_plot_liana_res.txt")
    
    write.csv(counts_cts_cond_r, file = filepath, row.names = FALSE)

}

## Heatmaps for interactions between celltypes

Heatmaps to visualize the number of connections between celltypes per condition/timepoint, directed. First unweighted, then weighted by magnitude of interactions.

In [ ]:
# Get data frames for directed heatmaps
# Get paths to import data frames as basis for heatmaps
circos_suffix = "_circos_plot_liana_res.txt"
path_glob_circos = os.path.join(path_out, "*" + circos_suffix) 
paths_circos = glob.glob(pathname=path_glob_circos)

min_nr = 0
max_nr_directed = float('-inf')

circos_csvs_directed = {}

for circos_csv in paths_circos:
    # Read in condition data frame
    cond = os.path.basename(circos_csv)[len(experiment_prefix)+1:-len(circos_suffix)]
    csv_cond = pd.read_csv(circos_csv)

    # Prepare data frame for heatmap plotting
    csv_pivoted = helpers.prep_heatmap_df(csv_cond, directed=True, weighted=False)
    circos_csvs_directed[cond] = csv_pivoted

    # Get max value of current df and update max_nr_directed
    max_cond_directed = csv_pivoted.max().max()
    max_nr_directed = max_cond_directed if max_cond_directed > max_nr_directed else max_nr_directed

In [ ]:
# Plot directed heatmaps
heatmaps = {}

for cond in liana_conds:
    heatmaps[cond] = helpers.plot_heatmap(circos_csvs_directed[cond], title=cond, vmin=min_nr, vmax=max_nr_directed)
    plt.show()


Define order of conditions for the heatmaps in one file and set number of columns and number of rows.


For `condtions` set an array in the shape of `[<condition 1>, <condition 2>, ...]` in the desired order. Condition names need to match the values in the condition column of the dataset. If conditions should be plotted in alphabetical order, ascending, set `condtions=liana_conds`.

'row_breaks' determines when the plotting skips to the next row in the figure. Set as is it assumes that all axes are to be filled with plots. If axes should be skipped, set `row_breaks=[<row number for break 1>, <row number for break 2 multiplied by column number>, ... ]`.

In [ ]:
n_cols = 3
n_rows = 1

conditions = ['00hr', '06hr', '24hr']

row_breaks = [n_rows * n for n in range(1, n_cols)]

In [ ]:
# Plot directed heatmaps in one figure
heatmaps_fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(40,10))

col_nr = 0
row_nr = 0 

for idx in range(len(conditions)):
    # Get condition name in predefined order
    cond = conditions[idx]

    #Breaks for switching to next column 
    if idx in row_breaks:
        col_nr += 1
        row_nr = 0

    helpers.plot_heatmap(circos_csvs_directed[cond], title=cond, vmin=min_nr, vmax=max_nr_directed, ax=axes[row_nr, col_nr] if n_rows > 1 else axes[col_nr])

    row_nr += 1

plt.tight_layout(pad=2.0, w_pad=2.0, h_pad=3.0)

plt.show()
plt.close()

In [ ]:
# Get dataframes for directed weighted heatmaps
lianas_weighted_directed = {}

min_score = 0
max_score_directed = float('-inf')

for cond in liana_conds:
    # Get liana results tables and filter for significance in magnitude and specificity of ligand-receptor interactions
    liana_res_cond = cond_adatas[cond].uns["liana_res"].copy()
    liana_res_cond = liana_res_cond[(liana_res_cond["magnitude_rank"] <= 0.05) & (liana_res_cond["specificity_rank"] <= 0.05)]
    
    # Get transformed heatmap for plotting
    liana_weighted = helpers.prep_heatmap_df(liana_res_cond, directed=True, weighted=True)
    lianas_weighted_directed[cond] = liana_weighted

    # Set max score for heatmaps
    cond_max_directed = liana_weighted.max().max()
    max_score_directed = cond_max_directed if cond_max_directed > max_score_directed else max_score_directed

In [ ]:
# Plot directed weighted heatmap
heatmaps_weighted = {}

for cond in liana_conds:
    heatmap = helpers.plot_heatmap(lianas_weighted_directed[cond], title=cond, vmin=min_score , vmax=max_score_directed)
    heatmaps_weighted[cond] = heatmap
    plt.show()

In [ ]:
# Plot directed weighted heatmaps in one figure
heatmaps_weighted_fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(40,10))

col_nr = 0
row_nr = 0 

for idx in range(len(conditions)):
    cond = conditions[idx]

    if idx in row_breaks:
        col_nr += 1
        row_nr = 0

    helpers.plot_heatmap(lianas_weighted_directed[cond], title=cond, vmin=min_score, vmax=max_score_directed, ax=axes[row_nr, col_nr] if n_rows > 1 else axes[col_nr])

    row_nr += 1

plt.tight_layout(pad=2.0, w_pad=2.0, h_pad=3.0)

plt.show()
plt.close()

Heatmaps to visualize the number of connections between celltypes per condition/timepoint, undirected. First unweighted, then weighted by magnitude of interactions.

In [ ]:
# Get data frames for undirected heatmaps
# Get paths to heatmaps
circos_suffix = "_circos_plot_liana_res.txt"
path_glob_circos = os.path.join(path_out, "*" + circos_suffix) 
paths_circos = glob.glob(pathname=path_glob_circos)

min_nr = 0
max_nr = float('-inf')

circos_csvs = {}

for circos_csv in paths_circos:
    # import heatmap data frame
    cond = os.path.basename(circos_csv)[len(experiment_prefix)+1:-len(circos_suffix)]
    csv_cond = pd.read_csv(circos_csv)

    # Prepare the data frame for heatmap plotting
    csv_mirrored = helpers.prep_heatmap_df(csv_cond, directed=False, weighted=False)
    circos_csvs[cond] = csv_mirrored

    # Get max value for count range
    max_cond = csv_mirrored.max().max()
    max_nr = max_cond if max_cond > max_nr else max_nr

In [ ]:
# Plot undirected heatmaps
heatmaps_undirected = {}
for cond in liana_conds:
    heatmap_undirected = helpers.plot_heatmap(circos_csvs[cond], title=cond, vmin=min_nr, vmax=max_nr)
    heatmaps_undirected[cond] = heatmap_undirected
    plt.show()

In [ ]:
# Plot undirected heatmaps in one figure
heatmaps_undirected_fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(40,10))

col_nr = 0
row_nr = 0 

for idx in range(len(conditions)):
    cond = conditions[idx]

    if idx in row_breaks:
        col_nr += 1
        row_nr = 0

    helpers.plot_heatmap(circos_csvs[cond], title=cond, vmin=min_nr, vmax=max_nr, ax=axes[row_nr, col_nr] if n_rows > 1 else axes[col_nr])

    row_nr += 1

plt.tight_layout(pad=2.0, w_pad=2.0, h_pad=3.0)

plt.show()
plt.close()

In [ ]:
# Get data frames for undirected weighted heatmaps 
lianas_weighted = {}

min_score = 0
max_score = float('-inf')

for cond in liana_conds:
    # Get liana results tables and filter for significance in magnitude and specificity of ligand-receptor interactions
    liana_res_cond = cond_adatas[cond].uns["liana_res"].copy()
    liana_res_cond = liana_res_cond[(liana_res_cond["magnitude_rank"] <= 0.05) & (liana_res_cond["specificity_rank"] <= 0.05)]

    # Get transformed data frame for heatmap plotting
    liana_pivoted = helpers.prep_heatmap_df(liana_res_cond, directed=False, weighted=True)
    lianas_weighted[cond] = liana_pivoted

    # Get max value from weighted data frame and set max_score
    cond_max = liana_pivoted.max().max()
    max_score = cond_max if cond_max > max_score else max_score

In [ ]:
# Plot undirected weighted heatmaps
heatmaps_undirected_weighted = {}

for cond in liana_conds:
    heatmap_undirected_weighted = helpers.plot_heatmap(lianas_weighted[cond], title=cond, vmin=min_score, vmax=max_score)
    heatmaps_undirected_weighted[cond] = heatmap_undirected_weighted
    plt.show()

In [ ]:
# Plot undirected weighted heatmaps in one figure
heatmaps_undirected_weighted_fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(40,10))

col_nr = 0
row_nr = 0

# Move to next column at row indexes [idx1, idx2, ...]
breaks_at = [n_rows * col for col in range(1, n_cols)]

for idx in range(len(conditions)):
    cond = conditions[idx]

    if idx in row_breaks:
        col_nr += 1
        row_nr = 0

    helpers.plot_heatmap(lianas_weighted[cond], title=cond, vmin=min_score, vmax=max_score, ax=axes[row_nr, col_nr] if n_rows > 1 else axes[col_nr])

    row_nr += 1

plt.tight_layout(pad=2.0, w_pad=2.0, h_pad=3.0)

plt.show()
plt.close()

#### Save heatmaps

In [ ]:
# Save directed heatmaps
for cond in heatmaps.keys():
    fig = heatmaps[cond].get_figure()
    for format in [".pdf", ".png"]:
        fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, cond, 'heatmap' + format])), bbox_inches='tight')

In [ ]:
# Save directed heatmaps in one figure
for format in [".pdf", ".png"]:
    heatmaps_fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, 'all_heatmap' + format])), bbox_inches='tight')

In [ ]:
# Save directed weighted heatmaps
for cond in heatmaps_weighted.keys():
    fig = heatmaps_weighted[cond].get_figure()
    for format in [".pdf", ".png"]:
        fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, cond, 'weighted_heatmap' + format])), bbox_inches='tight')

In [ ]:
# Save directed weighted heatmaps in one figure
for format in [".pdf", ".png"]:
    heatmaps_weighted_fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, 'all_weighted_heatmap' + format])), bbox_inches='tight')

In [ ]:
# Save undirected heatmaps
for cond in heatmaps_undirected.keys():
    fig = heatmaps_undirected[cond].get_figure()
    for format in [".pdf", ".png"]:
        fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, cond, 'undirected_heatmap' + format])), bbox_inches='tight')

In [ ]:
# Save undirected heatmaps in one figure
for format in [".pdf", ".png"]:
    heatmaps_undirected_fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, 'all_undirected_heatmap' + format])), bbox_inches='tight')

In [ ]:
# Save undirected weighted heatmaps
for conds in heatmaps_undirected_weighted.keys():
    fig = heatmaps_undirected_weighted[cond].get_figure()
    for format in [".pdf", ".png"]:
        fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, cond, 'undirected_weighted_heatmap' + format])), bbox_inches='tight')

In [ ]:
# Save undirected weighted heatmaps in one figure
for format in [".pdf", ".png"]:
    heatmaps_undirected_weighted_fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, 'all_undirected_weighted_heatmap' + format])), bbox_inches='tight')

#### Save data frames for heatmaps

In [ ]:
# Save datafrimes for directed heatmaps
for cond in circos_csvs_directed.keys():
    circos_csvs_directed[cond].to_csv(path_or_buf=os.path.join(path_out, "_".join([experiment_prefix, cond, "heatmap.txt.gz"])))

In [ ]:
# Save data frames for directed weighted heatmaps
for cond in lianas_weighted_directed.keys():
    lianas_weighted_directed[cond].to_csv(path_or_buf=os.path.join(path_out, "_".join([experiment_prefix, cond, "weighted_heatmap.txt.gz"])))

In [ ]:
# Save data frames for undirected heatmaps
for cond in circos_csvs.keys():
    circos_csvs[cond].to_csv(path_or_buf=os.path.join(path_out, "_".join([experiment_prefix, cond, "undirected_heatmap.txt.gz"])))

In [ ]:
# Save data frames for undirected weighted heatmaps
for cond in lianas_weighted.keys():
    lianas_weighted[cond].to_csv(path_or_buf=os.path.join(path_out, "_".join([experiment_prefix, cond, "undirected_weighted_heatmap.txt.gz"])))

## Normalized interactions plottet

Starting with unweighted interaction counts.

In [ ]:
# Get data frames with unfiltered ligand-receptor interactions per cell type pair
csv_conds_unfiltered = {}

for cond in liana_conds:
    # Get liana results tables and filter for significance in magnitude and specificity of ligand-receptor interactions
    liana_res_cond = cond_adatas[cond].uns["liana_res"].copy()
    #liana_res_cond = liana_res_cond[(liana_res_cond["magnitude_rank"] <= 0.05) & (liana_res_cond["specificity_rank"] <= 0.05)]
    
    # Get number of significant interactions per celltype pair
    counts_cts_cond = liana_res_cond.groupby(["source", "target"]).count()["ligand_complex"].reset_index()
    # Rename colums for R function
    counts_cts_cond_r = counts_cts_cond.rename(columns={"source": "source_cluster", 
                                                        "target": "target_cluster", 
                                                        "ligand_complex": "number"})
    
    csv_conds_unfiltered[cond] = counts_cts_cond_r

In [ ]:
for cond in liana_conds:
    print("Number of ligand-receptor interactions per cell type pair for condition " + cond + ": " + str(csv_conds_unfiltered[cond]['number'].unique()))

In [ ]:
# Get data frames with filtered ligand-receptor interactions per cell type pair
# Get paths to import data frames as basis for heatmaps
circos_suffix = "_circos_plot_liana_res.txt"
path_glob_circos = os.path.join(path_out, "*" + circos_suffix) 
paths_circos = glob.glob(pathname=path_glob_circos)

csv_conds = {}

for circos_csv in paths_circos:
    # Read in condition data frame
    cond = os.path.basename(circos_csv)[len(experiment_prefix)+1:-len(circos_suffix)]
    csv_cond = pd.read_csv(circos_csv)

    # Add to dict
    csv_conds[cond] = csv_cond

In [ ]:
# Get data frames with normalized ligand-receptor interactions per cell type pair (# filtered interactions/# unfiltered interactions)
csv_conds_norm = {}

for cond in liana_conds:
    csv_conds_norm[cond] = csv_conds[cond].drop(columns=['number'])
    csv_conds_norm[cond]['number'] = csv_conds[cond]['number'] / csv_conds_unfiltered[cond]['number']

### Heatmaps with normalized interactions

Heatmaps to visualize the number of connections between celltypes per condition/timepoint normalized against the total number of ligand-receptor pairs in the data set, directed. First unweighted.

In [ ]:
# Get data frames for directed heatmaps with normalized values
min_nr_norm = 0
max_nr_directed_norm = float('-inf')

circos_csvs_directed_norm = {}

for cond in liana_conds:
    # Prepare data frame for heatmap plotting
    csv_pivoted_norm = helpers.prep_heatmap_df(csv_conds_norm[cond], directed=True, weighted=False)
    circos_csvs_directed_norm[cond] = csv_pivoted_norm

    # Get max value of current df and update max_nr_directed
    max_cond_directed_norm = csv_pivoted_norm.max().max()
    max_nr_directed_norm = max_cond_directed_norm if max_cond_directed_norm > max_nr_directed_norm else max_nr_directed_norm

In [ ]:
# Plot directed heatmaps
heatmaps_norm = {}

for cond in liana_conds:
    heatmaps_norm[cond] = helpers.plot_heatmap(circos_csvs_directed_norm[cond], title=cond, vmin=min_nr_norm, vmax=max_nr_directed_norm)
    plt.show()

Define order of conditions for the heatmaps in one file and set number of columns and number of rows.


For `condtions` set an array in the shape of `[<condition 1>, <condition 2>, ...]` in the desired order. Condition names need to match the values in the condition column of the dataset. If conditions should be plotted in alphabetical order, ascending, set `condtions=liana_conds`.

'row_breaks' determines when the plotting skips to the next row in the figure. Set as is it assumes that all axes are to be filled with plots. If axes should be skipped, set `row_breaks=[<row number for break 1>, <row number for break 2 multiplied by column number>, ... ]`.

In [ ]:
n_cols = 3
n_rows = 1

conditions = ['00hr', '06hr', '24hr']


row_breaks = [n_rows * n for n in range(1, n_cols)]

In [ ]:
# Plot directed heatmaps in one figure
heatmaps_fig_norm, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(40,10))

col_nr = 0
row_nr = 0 

for idx in range(len(conditions)):
    # Get condition name in predefined order
    cond = conditions[idx]

    #Breaks for switching to next column 
    if idx in row_breaks:
        col_nr += 1
        row_nr = 0

    helpers.plot_heatmap(circos_csvs_directed_norm[cond], title=cond, vmin=min_nr_norm, vmax=max_nr_directed_norm, ax=axes[row_nr, col_nr] if n_rows > 1 else axes[col_nr])

    row_nr += 1

plt.tight_layout(pad=2.0, w_pad=2.0, h_pad=3.0)

plt.show()
plt.close()

Heatmaps to visualize the number of connections between celltypes per condition/timepoint normalized against total number of ligand-receptor pairs in the data set, undirected. First unweighted.

In [ ]:
# Get data frames for undirected heatmaps with normalized values
min_nr_norm = 0
max_nr_norm = float('-inf')

circos_csvs_norm = {}

for cond in liana_conds:
    # Prepare the data frame for heatmap plotting
    csv_mirrored_norm = helpers.prep_heatmap_df(csv_conds_norm[cond], directed=False, weighted=False)
    circos_csvs_norm[cond] = csv_mirrored_norm

    # Get max value for count range
    max_cond_norm = csv_mirrored_norm.max().max()
    max_nr_norm = max_cond_norm if max_cond_norm > max_nr_norm else max_nr_norm

In [ ]:
# Plot undirected heatmaps
heatmaps_undirected_norm = {}

for cond in liana_conds:
    heatmap_undirected_norm = helpers.plot_heatmap(circos_csvs_norm[cond], title=cond, vmin=min_nr_norm, vmax=max_nr_norm)
    heatmaps_undirected_norm[cond] = heatmap_undirected_norm
    plt.show()

In [ ]:
# Plot undirected heatmaps in one figure
heatmaps_undirected_fig_norm, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(40,10))

col_nr = 0
row_nr = 0 

for idx in range(len(conditions)):
    cond = conditions[idx]

    if idx in row_breaks:
        col_nr += 1
        row_nr = 0

    helpers.plot_heatmap(circos_csvs_norm[cond], title=cond, vmin=min_nr_norm, vmax=max_nr_norm, ax=axes[row_nr, col_nr] if n_rows > 1 else axes[col_nr])

    row_nr += 1

plt.tight_layout(pad=2.0, w_pad=2.0, h_pad=3.0)

plt.show()
plt.close()

#### Save heatmaps

In [ ]:
# Save directed heatmaps
for cond in heatmaps_norm.keys():
    fig = heatmaps_norm[cond].get_figure()
    for format in [".pdf", ".png"]:
        fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, cond, 'heatmap_normalized' + format])), bbox_inches='tight')

# Save directed heatmaps in one figure
for format in [".pdf", ".png"]:
    heatmaps_fig_norm.savefig(os.path.join(path_out, '_'.join([experiment_prefix, 'all_heatmap_normalied' + format])), bbox_inches='tight')

# Save undirected heatmaps
for cond in heatmaps_undirected_norm.keys():
    fig = heatmaps_undirected_norm[cond].get_figure()
    for format in [".pdf", ".png"]:
        fig.savefig(os.path.join(path_out, '_'.join([experiment_prefix, cond, 'undirected_heatmap_normalized' + format])), bbox_inches='tight')

# Save undirected heatmaps in one figure
for format in [".pdf", ".png"]:
    heatmaps_undirected_fig_norm.savefig(os.path.join(path_out, '_'.join([experiment_prefix, 'all_undirected_heatmap_normalized' + format])), bbox_inches='tight')

#### Save heatmap data frames

In [ ]:
# Save datafrimes for directed heatmaps
for cond in circos_csvs_directed.keys():
    circos_csvs_directed[cond].to_csv(path_or_buf=os.path.join(path_out, "_".join([experiment_prefix, cond, "heatmap_normalized.txt.gz"])))

# Save data frames for undirected heatmaps
for cond in circos_csvs.keys():
    circos_csvs[cond].to_csv(path_or_buf=os.path.join(path_out, "_".join([experiment_prefix, cond, "undirected_heatmap_normalized.txt.gz"])))

## Plots showing difference in number of interactions between celltypes between conditions/timepoints.

Choose one condition/timepoint to look at. Compare the difference in LIANA detected CCC events in relation to each of the other conditions/timepoints.

### Diverging Barplots

In [ ]:
%%R -i liana_conds -i path_out

barplots <- c()
barplot_dfs <- c()

for (condition1 in liana_conds) {
    for (condition2 in liana_conds){
        # Don't do the comparison if condition one and two are the same
        if (condition1 == condition2) next
        
        # Get data frame with differences for plotting
        result <- prep_data_barplot(counts_conds[[condition1]], counts_conds[[condition2]])
        
        # Set plot title
        plot_title <- paste0("Changes in number of CCC events between celltypes in condition \n", condition1, " vs ", condition2)

        # Create the plot
        p <- visualize_differences(result, title=plot_title)
        
        vs = paste0(condition1, "_vs_", condition2)
        
        barplots[[vs]] <- p
        barplot_dfs[[vs]] <- result
        
        print(p)
    }
}


#### Save Barplots.

In [ ]:
%%R -i path_out -i experiment_prefix

for (name in names(barplots)) {
    # Set flexible file width and height
    file_width <- max(max(nchar(barplot_dfs[[name]]$pair_label)) / 2, 14)
    file_height <- max(length(barplot_dfs[[name]]$source_cluster) / 4, 12)

    # Save plot as PNG
    filepath <- paste0(path_out, experiment_prefix, "_", name, "_diverging_barplot.png")
    ggsave(filepath, plot=barplots[[name]], width=file_width, height=file_height, units="in", dpi=300, limitsize=FALSE)

    # Save plot as PDF
    filepath <- paste0(path_out, experiment_prefix, "_", name, "_diverging_barplot.pdf")
    ggsave(filepath, plot=barplots[[name]], width=file_width, height=file_height, units="in", dpi=300, limitsize=FALSE)
}

#### Save tables for Barplots.

In [ ]:
%%R -i path_out -i experiment_prefix

for (name in names(barplot_dfs)) {       
    # Set path for data frame saving
    filepath <- paste0(path_out, experiment_prefix, "_", name, "_diverging_barplot.txt.gz")
    # Save data frame
    write.csv(barplot_dfs[[name]][, c("source_cluster", "target_cluster", "number")], filepath, row.names = FALSE)
}

# Save condition AnnDatas

Save the AnnData objects for the conditions/timepoints with the ligand-receptor inference information to .h5ad files.

In [ ]:
for cond in liana_conds:
    cond_adatas[cond].write_h5ad(filename=os.path.join(path_out, "_".join([experiment_prefix, cond, "liana_results.h5ad"])))

# Save experiment AnnData

Save full experiment AnnData object with all condition liana results tables.

In [ ]:
# Make sure the original X matrix of the AnnData is restored
if layer_raw == "X":
    adata.X = adata.raw.X.copy()
    del adata.raw
else:
    adata.X = adata.layers[".X"]
    del adata.layers[".X"]

# Add max_col and max_size to Anndata
adata.uns['max_col_liana'] = max_col
adata.uns['max_size_liana'] = max_size
    
# Get filename from path_in_adata and edit for saveing new adata
file_name = "_".join([experiment_prefix, "liana_results.h5ad"])

# Save adata as .h5ad file to out path
adata.write_h5ad(filename=os.path.join(path_out, file_name))

# Session info

Show session info for reproducability.

In [ ]:
%%R
sessionInfo()

In [ ]:
session_info.show()

# Save notebook

In [ ]:
%notebook -e # path_out -> doesn't recognise path_out as variable